<a href="https://colab.research.google.com/github/emerenan/xlsx_validation/blob/main/cz_comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
!pip install xlsxwriter
import xlsxwriter
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill, Font
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule
from unidecode import unidecode

TowerDB Comparison

In [18]:
def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save,old_file,new_file, type_file='mix',kind='tw', sheetname='', skipr=0, skipc=0):
    import re
    import xlsxwriter
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = path_OLD
        df_OLD = fit_df(df_OLD)

        df_NEW = path_NEW
        df_NEW = fit_df(df_NEW)
        """df_NEW[index_col] = df_NEW[index_col].astype(str)
        df_NEW = df_NEW.drop_duplicates(subset=index_col)
        df_NEW['sites'] = df_NEW[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')"""

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status}_current'
        status_2 = f'{status}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}_{old_file} vs {new_file}.xlsx'
    #file = pd.ExcelWriter(fname, engine='openpyxl')
    file = pd.ExcelWriter(fname, engine='xlsxwriter')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)
    
    file.save()

    # get openpyxl objects
    from openpyxl import load_workbook

    wb  = load_workbook(fname)
    ws = wb['diffs_founded']
    # get openpyxl objects
    """wb  = file.book
    ws = file.sheets['diffs_founded']"""
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ20000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
        
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ20000', header_style)
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

def read_files(path, sheetname, n_skiprows, n_skip_columns, site_index, cols_number, col_dates, format='normal', type_date="%d/%m/%Y"):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ']
        #lista = []

        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(0)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    date_parser(df, col_dates, format, type_date)
    # define the entiry columns which doesn't have NaN 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_number)

    for col in cols_number:
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    
    df = df.fillna('')

    return df

path_towerdb = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
towerdb_col = ["Code (Duplicate)","Site Status","VF - In scope / out of scope (Generalised scoping)","Site in Skylon scope Actual (From Site List Sheet )",\
               "Legacy Site Code(Duplicate)","TIMS Site Code","Legacy Site Code","Site Name","Macro Region","Province","Municipality","Inhabitants","Address",\
               "Ground Register","Altitude","Latitude","Longitude","Categorization by Inhabitants","Categorization by Transmission Sys",\
               "Categorization by Site Type","Categorization by Transmission Sys (subcluster)","Other internal Categorization 1 (Identify ACQ Sites)",\
               "Other internal Categorization 2 Energy provider (Eon/ LL)","DAS+Macro","DAS (Yes/ No)","DAS Ownership (Complete/ Partial/ 3rd Party)",\
               "Active/ Passive DAS","# of remote units/ radiating points","Type of Structure","Distance highest antenna to ground level","GBT Tower height",\
               "POD ID","Energy Consumption LTM (kwh)","Annual Energy cost LTM (Euros)","Infrastructure ready (existing)/ to be ready (new)",\
               "Infrastructure to be dismantled by","Radio equipments to be deactivated by","Infrastructure to be shared by","Technology VOD","Fibre / Microwave",\
               "Vertical passive structure owner","Room configuration (detailed)","Shelter passive structure ownership","Type of Air Conditioning",\
               "Number of cabinets (Full Capacity)","Number of Antenna (Full Capacity)","Number of MW (Full Capacity)","Counterpart","# of Lease Contracts",\
               "Current annual lease fees ","Current other fees (Maintenance)","Current other fees","(Average) residual duration - Lease contract",\
               "(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both","# of Tenants Agreements",\
               "Current Total Annual Hosting Fees","Tenant (name/ID) MNO1 (Česká telekomunikační infrastruktura a.s.)","Annual Fee per Tenant MNO1",\
               "Annual Energy Fee MNO1","Annual Maintenance Fee MNO1","Other Services Fee MNO1","Residual duration MNO1 (Years)",\
               "Tenant (name/ID) MNO2 (T-Mobile Czech Republic a.s.)","Annual Fee per Tenant MNO2","Annual Energy Fee MNO2","Annual Maintenance Fee MNO2",\
               "Other Services Fee MNO2","Residual duration MNO2 (days)","Tenant (name/ID) MNO3","Annual Fee per Tenant MNO3","Annual Energy Fee MNO3",\
               "Annual Maintenance Fee MNO3","Other Services Fee MNO3","Residual duration MNO3","# of OTMOs","Annual Fee from OTMOs","Annual Energy Fee from OTMOs",\
               "Annual Maintenance Fee OTMOs","Other Services Fee OTMOs","Average residual duration (days)","Check","Strategic Macro Sites","Critical Sites",\
               "Case A Core Site","Macro Site - Transmission Hub Site","Macro Site - Transmission Hub Site with/without Shelters","Transmission Sites",\
               "Room Configuration","Power Supply","Air Conditioning","Active Sharing Arrangements involving the Operator","VF-CZ Demerger phase",\
               "EVO Location [FAR Site ID] ","Billing Trigger date ","Strategic_Site_Bucket","Critical Site Bucket","Subsequent_Sharing_Arrangement",\
               "First_Active_Sharing_Deployment_Type","First_Active_Sharing_Start_Date","First_Active_Sharing_End_Date","Decommissioned_Site","Wip_Site",\
               "Bts_Site","Transfer_Date_Of_Consent_Required_Sites","Sites_As_Metered_Estimated","Date_Of_Equipment_Removal"]
towerdb_col = lower_str(towerdb_col)

dates = ['Billing Trigger date ', "Infrastructure ready (existing)/ to be ready (new)", "Infrastructure to be dismantled by",\
         'First_Active_Sharing_Start_Date','First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
         'Date_Of_Equipment_Removal']
num_parse = ["Inhabitants", "Current annual lease fees ","(Average) residual duration - Lease contract",\
             "(Average) residual duration - Maintenance contract","(Average) residual duration - Lease & Maintenance both",\
             "# of Tenants Agreements","Current Total Annual Hosting Fees", "Annual Fee per Tenant MNO1",\
             "Annual Energy Fee MNO1", "Annual Fee per Tenant MNO2","Annual Energy Fee MNO2",\
             "Residual duration MNO2 (days)", "Annual Fee from OTMOs","Annual Energy Fee from OTMOs",\
             "Other Services Fee OTMOs","Average residual duration (days)"]

tab = 'Tower DB'
row = 3
towerdb = read_files(path_towerdb,tab, 3, 0,"Code (Duplicate)",num_parse,dates)
towerdb.columns = lower_str(list(towerdb.columns))
towerdb = towerdb[towerdb_col]

path_msa = '/content/TowerDB_CzechRepublic_20210731 (3).csv'
msa = pd.read_csv(path_msa,encoding='ISO-8859-1').fillna('')
msa_cols = lower_str(list(msa.columns))
msa.columns = msa_cols
msa = msa.rename(columns={'ï»¿code (duplicate)': 'code (duplicate)'})

bill_col= ['Code (Duplicate)',\
            "Site Status",\
            "Categorization by Transmission Sys",\
            "Categorization by Site Type",\
            'Strategic Macro Sites',\
            'Critical Sites',\
            'Case A Core Site',\
            'Macro Site - Transmission Hub Site',\
            'Macro Site - Transmission Hub Site with/without Shelters',\
            'Transmission Sites',\
            'Room Configuration',\
            'Power Supply',\
            'Air Conditioning',\
            'Active Sharing Arrangements involving the Operator',\
            'VF-CZ Demerger phase',\
            "Billing Trigger date ",\
            'Strategic_Site_Bucket',\
            'Critical Site Bucket',\
            'Wip_Site',\
            'Bts_Site',\
            'Sites_As_Metered_Estimated',\
            'Subsequent_Sharing_Arrangement',\
            "First_Active_Sharing_Deployment_Type",\
            "First_Active_Sharing_Start_Date",\
            "First_Active_Sharing_End_Date",\
            "Decommissioned_Site",\
            "Transfer_Date_Of_Consent_Required_Sites",\
            "Date_Of_Equipment_Removal"]

tw_save = '/content/CZ_TW'
old_n = 'twdb_20210831 (2).csv'
new_n = 'twdb_celfinet_20210831.csv'


find_diffs_between_files(msa, towerdb, 'Code (Duplicate)', bill_col, \
                             "Site Status", tw_save,old_n,new_n, type_file='else',kind='tw')


In [ ]:
def find_diffs_between_files_1(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save, old_name,new_name, type_file='mix',status_col='', \
                             kind='tw', dates_cols=[], kind_col='',type_date="%d/%m/%Y", sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, index_col, kind, kind_col):
        kind_col = kind_col.lower()
        kind = kind.lower()
        if kind == 'ta':
            df = df.dropna(subset=[index_col], axis=0)
            df['sites'] = df[index_col].astype(str) + df[kind_col] 
            #df.columns = lower_str(list(df.columns))
            df = df.dropna(subset=['sites'], axis=0)
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        else:
            """for col in cols_conv:
                df[col] = df[col].astype(str).apply(unidecode)"""
            df = df.dropna(subset=[index_col], axis=0)
            df = df.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
            #df = df.drop_duplicates(subset=[index_col], keep='last')
            df[index_col] = df[index_col].astype(str)
            df['sites'] = df[index_col]
            # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
            df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def csv_header(path):
        """Função usada para ler os ficheiros que tem o simbolo do Euro"""
        import csv
        f = open(path, encoding='windows-1252', errors='ignore')
        data = []
        for row in csv.reader(f, delimiter=','):
            data.append(row)
        col = lower_str([*data[0]])
        #data.pop(0)
        #df = pd.DataFrame(data, columns=col)
        return  col

    def comparison(df_old, df_new,status):
        # Perform Diff
        new_copy = df_NEW.copy()
        droppedRows = []
        newRows = []

        cols_OLD = list(df_OLD.columns)
        cols_NEW = list(df_NEW.columns)
        sharedCols = list(set(cols_OLD).intersection(cols_NEW))
        #print(sharedCols)
        for row in new_copy.index:
            if (row in df_OLD.index) and (row in df_NEW.index):
                for col in sharedCols:
                    value_OLD = str(df_OLD.loc[row,col])
                    value_NEW = str(df_NEW.loc[row,col])
                #Error de a.empty() são sites duplcados e nã poodem estar no index
                    if value_OLD == value_NEW:
                        new_copy.loc[row,col] = np.nan
                    else:
                        new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
            else:
                newRows.append(row)

        new_copy = new_copy.dropna(axis=0, how='all')
        new_copy = new_copy.dropna(axis=1, how='all')

        for row in df_OLD.index:
            if row not in df_NEW.index:
                droppedRows.append(row)
                new_copy = new_copy.append(df_OLD.loc[row,:])
        
        new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
        
        new_copy = new_copy.reset_index()

        if kind=='tw':
            sites = [i for i in new_copy['sites']]
            old = df_OLD[[status]].reset_index()
            old = old.loc[old['sites'].isin(sites)]
            new = df_NEW[[status]].reset_index()
            new = new.loc[new['sites'].isin(sites)]
            df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
            new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
            status_1 = f'{status}_current'
            status_2 = f'{status}_before'
            new_copy = new_copy.set_index('sites')
            new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
            new_copy = new_copy.reset_index()

        return newRows, droppedRows, new_copy

    def date_parser(df, columns, format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    #cols_conv = lower_str(cols_conv)
    dates_cols = lower_str(dates_cols)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD = df_OLD.iloc[:,skipc:]
        date_parser(df_OLD, dates_cols,format , type_date)
        df_OLD = fit_df(df_OLD,index_col,kind, kind_col)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols,format , type_date)
        df_NEW = fit_df(df_NEW,index_col,kind, kind_col)

    elif type_file == 'csv':
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        df_OLD = pd.read_csv(path_OLD,encoding='ISO-8859-1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col,cols_conv, kind, kind_col)
        for col in ['province','municipality', 'address']:
            df_OLD[col] = df_OLD[col].apply(unidecode)

        #cols_new = csv_header(path_NEW)
        #Se for o arquivo gerado a partir do xlsx não prcisa de encoding
        df_NEW = pd.read_csv(path_NEW,encoding='ISO-8859-1').fillna('')
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = fit_df(df_NEW, index_col,cols_conv, kind, kind_col)
        for col in ['province','municipality', 'address']:
            df_NEW[col] = df_NEW[col].apply(unidecode)

    else:
        #cols_old = csv_header(path_OLD) header=0, names=cols_old,
        df_OLD = pd.read_csv(path_OLD,encoding='ISO-8859-1').fillna('')
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = fit_df(df_OLD, index_col, kind, kind_col)
        for col in ['province','municipality', 'address']:
            df_OLD[col] = df_OLD[col].apply(unidecode)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        #print(df_NEW.columns)
        df_NEW.columns = lower_str(list(df_NEW.columns)) 
        df_NEW = df_NEW.iloc[:,skipc:]
        date_parser(df_NEW, dates_cols,format , type_date)
        df_NEW = fit_df(df_NEW, index_col, kind, kind_col)
        for col in ['province','municipality', 'address']:
            df_NEW[col] = df_NEW[col].apply(unidecode)

    newRows, droppedRows, df_all = comparison(df_OLD, df_NEW, status_col)

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - ({old_name}) vs ({new_name}).xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    df_all.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                            end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)
    
    #print(len(newRows))
    for site in df_all['sites']:
        if site in newRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = df_all.index[df_all[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)

    wb.save(fname)
    print('\nDone.\n')

bill_col= ['Code (Duplicate)',\
            "Site Status",\
            "Categorization by Transmission Sys",\
            "Categorization by Site Type",\
            'Strategic Macro Sites',\
            'Critical Sites',\
            'Case A Core Site',\
            'Macro Site - Transmission Hub Site',\
            'Macro Site - Transmission Hub Site with/without Shelters',\
            'Transmission Sites',\
            'Room Configuration',\
            'Power Supply',\
            'Air Conditioning',\
            'Active Sharing Arrangements involving the Operator',\
            'VF-CZ Demerger phase',\
            "Billing Trigger date ",\
            'Strategic_Site_Bucket',\
            'Critical Site Bucket',\
            'Wip_Site',\
            'Bts_Site',\
            'Sites_As_Metered_Estimated',\
            'Subsequent_Sharing_Arrangement',\
            "First_Active_Sharing_Deployment_Type",\
            "First_Active_Sharing_Start_Date",\
            "First_Active_Sharing_End_Date",\
            "Decommissioned_Site",\
            "Transfer_Date_Of_Consent_Required_Sites",\
            "Date_Of_Equipment_Removal"]

pathtw = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
pathold = '/content/TowerDB_CzechRepublic_20210731 (3).csv'
tw_save = '/content/CZ_TW'
tab = 'Tower DB'
row = 3
old_n = 'twdb_20210831 (2).csv'
new_n = 'twdb_celfinet_20210831.csv'
cols_con = ['Province','Municipality','Address']
dates = ['Billing Trigger date ','infrastructure ready (existing)/ to be ready (new)',\
         'First_Active_Sharing_Start_Date',\
         'First_Active_Sharing_End_Date', 'Transfer_Date_Of_Consent_Required_Sites',\
         'Date_Of_Equipment_Removal']  

find_diffs_between_files_1(pathold, pathtw, 'Code (Duplicate)', bill_col,tw_save, old_n,new_n,'mix',\
                           status_col="Site Status" , kind='tw', dates_cols=dates , kind_col='', sheetname=tab, skipr = row)

# TA Input Comparisons


CSV

In [ ]:
def find_diffs_between_files(path_OLD, path_NEW,new_file,old_file, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', kind_col='', k='', am_cols = [],\
                             cols_int=[], cols_date=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist
     
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format
    
    def date_parser(df, column,format, type_date):
        if format == 'mix':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
            return df[column]
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df


    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']
        ta_cols = lower_str(ta_cols)
        df_OLD = pd.read_csv(path_OLD, header=0, names=ta_cols)
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD = df_OLD.iloc[natsort.index_humansorted(df_OLD['tenant agreement id - new'])]
        
        #Ordenando as duplicatas
        old_lista=[]
        dico = {}
        for id in df_OLD['tenant agreement id - new']:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                old_lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                old_lista.append(n)
        print(old_lista)
        df_OLD['sites'] = old_lista
        print(df_OLD['sites'])
        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD['site name']+df_OLD['count of contracts'].astype(str)+df_OLD['service type'].astype(str)
        df_OLD = df_OLD.fillna('')
        for in_col in ['counterpart','annual amount in czk', 'amount in eur', 'index', 'counterpart_1']:
            df_OLD[in_col] = df_OLD[in_col].apply(unidecode)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_OLD = df_OLD.set_index('sites').fillna('')

        """New File"""
        df_NEW = pd.read_csv(path_NEW, header=0, names=ta_cols)
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_NEW = df_NEW.iloc[natsort.index_humansorted(df_NEW['tenant agreement id - new'])]
        
        #Ordenando as duplicatas
        new_lista=[]
        dic = {}
        for id in df_NEW['tenant agreement id - new']:
            nome = str(id)
            if nome not in dic.keys():
                dic[nome] = 1
                n = nome+f'_{dic[nome]}'
                new_lista.append(n)
            else:
                dic[nome] += 1
                n = nome+f'_{dic[nome]}'
                new_lista.append(n)
        print(new_lista)
        df_NEW['sites'] = new_lista
        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD['site name']+df_OLD['count of contracts'].astype(str)+df_OLD['service type'].astype(str)
        df_NEW = df_NEW.fillna('')
        for in_col in ['counterpart','annual amount in czk', 'amount in eur', 'index', 'counterpart_1']:
            df_NEW[in_col] = df_NEW[in_col].apply(unidecode)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')

    else:
        ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']
        ta_cols = lower_str(ta_cols)
        df_OLD = pd.read_csv(path_OLD, header=0, names=ta_cols)
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD = df_OLD.iloc[natsort.index_humansorted(df_OLD['tenant agreement id - new'])]
        
        #Ordenando as duplicatas
        old_lista=[]
        dico = {}
        for id in df_OLD['tenant agreement id - new']:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                old_lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                old_lista.append(n)
        print(old_lista)
        df_OLD['sites'] = old_lista
        print(df_OLD['sites'])
        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD['site name']+df_OLD['count of contracts'].astype(str)+df_OLD['service type'].astype(str)
        df_OLD = df_OLD.fillna('')
        for in_col in ['counterpart','annual amount in czk', 'amount in eur', 'index', 'counterpart_1']:
            df_OLD[in_col] = df_OLD[in_col].apply(unidecode)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_OLD = df_OLD.set_index('sites').fillna('')

        col = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type',\
                'Contract start date', 'Contract end date', 'Status of contract',\
                'Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart',\
                'Classification of Tenant', 'Annual amount - billing currency',\
                'Billing Currency', 'Annual Amount in CZK', 'Amount in EUR',\
                'Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
                'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key',\
                'Classification', 'Residual Period in Years', 'Remarks',\
                'Count of unique key', 'Count of contracts', 'Scoping classification',\
                'DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
                'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x',\
                'SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count','Quota of Unique Key','unamed']
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,header=0, names=col, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=ta_cols)
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW = df_NEW.iloc[natsort.index_humansorted(df_NEW['tenant agreement id - new'])]
        
        #Ordenando as duplicatas
        new_lista=[]
        dicn = {}
        for id in df_NEW['tenant agreement id - new']:
            nome = str(id)
            if nome not in dicn.keys():
                dicn[nome] = 1
                n = nome+f'_{dicn[nome]}'
                new_lista.append(n)
            else:
                dicn[nome] += 1
                n = nome+f'_{dicn[nome]}'
                new_lista.append(n)
        print(new_lista)
        df_NEW['sites'] = new_lista
        #df_NEW['sites'] = df_NEW[index_col].astype(str)+df_NEW[kind_col].astype(str)+df_NEW['site name']+df_NEW['count of contracts'].astype(str)+df_NEW['service type'].astype(str)
        #print([i for i in list(df_NEW['sites']) if list(df_NEW['sites']).count(i)>1])
        df_NEW = replace_values(df_NEW, am_cols, 0)
        for col in am_cols:
            #df[col] = df[col].fillna(0)
            df_NEW[col] = df_NEW[col].astype(int).apply(lambda x: f'{x:,}')

        for col in cols_int:
            df_NEW[col] = df_NEW[col].fillna(0)
            df_NEW[col] = df_NEW[col].astype(int)

        df_NEW = df_NEW.fillna('')
        for in_col in ['counterpart','annual amount in czk', 'amount in eur', 'index', 'counterpart_1']:
            df_NEW[in_col] = df_NEW[in_col].apply(unidecode)
        df_NEW = replace_values(df_NEW,bill_cols)
        for i in cols_date:
            df_NEW[i] = date_parser(df_NEW, i, 'mix', "%d/%m/%Y")
        df_NEW = df_NEW.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

        try:
            df_NEW = df_NEW.set_index('sites').fillna('')
        except:
            print('falha')
        #print(list(df_NEW.columns)==(df_OLD.columns))

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    
    n = 10
    lista = []
    for site in new_copy['sites']:
        chunks = [site[i:i+n] for i in range(0, len(site), n)]
        lista.append(chunks[0])
    new_copy['sites_code'] = lista
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} {old_file} vs {new_file}.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    ws.delete_cols(2)
    wb.save(fname)
    print('\nDone.\n')

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist
ta_cols = ['Tenant Agreement ID - NEW', 'Code','Contract start date', 'Contract end date','Counterpart',\
           'Classification of Tenant',]
path_ta_input = '/content/TA_Input_CzechRepublic_20210831.csv'
sheet = 'Tenant '
ta_old = '/content/TA_Input_CzechRepublic_20210831 (2).csv'
ta_save = '/content/CZ_TA'
#find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, "", ta_save,'csv', 'ta')

#cols_con_ta = ['Province','Municipality','Address']
dates = lower_str(['Contract start date', 'Contract end date', 'Update in month'])
amount_ta = lower_str(['Annual Amount in CZK', 'Amount in EUR'])
interger = lower_str(['annual amount - billing currency','Residual Period in Years','Count of unique key', 'Count of contracts', 'Unique Tenant Count'])
ta_bill_cols = lower_str(['Tenant Agreement ID - NEW','Contract start date','Contract end date',\
                'Counterpart','Classification of Tenant'])
ta_nn='Celfinet_20210831.csv'
ta_on='TA_20210831 (2).csv'

find_diffs_between_files(ta_old, path_ta_input, ta_nn, ta_on, 'Code', ta_cols, status_col='', path_save=ta_save, type_file='csv',kind='ta',\
                         kind_col='tenant agreement id - new',k='service type',am_cols = amount_ta, cols_int=interger, cols_date=dates, sheetname=sheet)

Excel

In [24]:
def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', kind_col='', k='', sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, bill_cols):
        df = df.dropna(subset=[index_col], axis=0)
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        ta_cols = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type','Contract start date', \
           'Contract end date', 'Status of contract','Renewal Option', 'Comments', 'Counterpart ID', \
           'Counterpart','Classification of Tenant', 'Annual amount - billing currency','Billing Currency', \
           'Annual Amount in CZK', 'Amount in EUR','Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
           'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key','Classification', \
           'Residual Period in Years', 'Remarks','Count of unique key', 'Count of contracts', \
           'Scoping classification','DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
           'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x','SiteType',\
           '1.028', 'Unique Tenant', 'Unique Tenant Count', 'unamed', 'unamed_2']
        ta_cols = lower_str(ta_cols)
        df_OLD = pd.read_csv(path_OLD, header=0, names=ta_cols, encoding='latin')
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #df_OLD['sites'] = [site if type(site) != str else len(df_OLD[index_col])+i for i, site in enumerate(df_OLD[index_col])]
        #df_OLD = df_OLD.sort_values(by='sites')
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD['site name']+df_OLD['count of contracts'].astype(str)+df_OLD['service type'].astype(str)
        

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_OLD = df_OLD.set_index('sites').fillna('')

        col = ['Tenant Agreement ID - NEW', 'Code', 'Site Name', 'Service Type',\
                'Contract start date', 'Contract end date', 'Status of contract',\
                'Renewal Option', 'Comments', 'Counterpart ID', 'Counterpart',\
                'Classification of Tenant', 'Annual amount - billing currency',\
                'Billing Currency', 'Annual Amount in CZK', 'Amount in EUR',\
                'Terms of Payment', 'Frequency', 'Indexed YES/NO', 'Index',\
                'Percentage', 'VAT Subject YES/NO', 'Percentage (VAT)', 'Unique Key',\
                'Classification', 'Residual Period in Years', 'Remarks',\
                'Count of unique key', 'Count of contracts', 'Scoping classification',\
                'DAS sites', 'DAS ownership', '31-Mar-21', 'Update in month',\
                'Updated Item', 'Comment', 'Counterpart ID_1', 'Counterpart_1', 'x',\
                'SiteType', '1.028', 'Unique Tenant', 'Unique Tenant Count','Quota of Unique Key','unamed']
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,header=0, names=col, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=ta_cols)
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW['sites'] = df_NEW[index_col].astype(str)+df_NEW[kind_col].astype(str)+df_NEW['site name']+df_NEW['count of contracts'].astype(str)+df_NEW['service type'].astype(str)
        #print([i for i in list(df_NEW['sites']) if list(df_NEW['sites']).count(i)>1])

        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

        try:
            df_NEW = df_NEW.set_index('sites').fillna('')
        except:
            print('falha')
        #print(list(df_NEW.columns)==(df_OLD.columns))

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]
                value_NEW = df_NEW.loc[row,col]
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    if kind=='tw':
        sites = [i for i in new_copy['sites']] 
        old = df_OLD[[status_col]].reset_index()
        old = old.loc[old['sites'].isin(sites)]
        new = df_NEW[[status_col]].reset_index()
        new = new.loc[new['sites'].isin(sites)]
        df_cross = pd.merge(new, old, on=['sites'], how='inner', suffixes=('_current', '_before'))
        new_copy = pd.merge(new_copy, df_cross, on=['sites'], how='left')
        status_1 = f'{status_col}_current'
        status_2 = f'{status_col}_before'
        new_copy = new_copy.set_index('sites')
        new_copy = new_copy[[status_1, status_2]+ new_copy.columns[:-2].tolist()]
        new_copy = new_copy.reset_index()


    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy[index_col]:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

ta_cols = ['Tenant Agreement ID - NEW', 'Code','Contract start date', 'Contract end date','Counterpart',\
           'Classification of Tenant',]
path_ta_input = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
sheet = 'Tenant '
ta_old = '/content/TA_Input_CzechRepublic_20210731.csv'
ta_save = '/content/CZ_TA'
old_ta = 'TA_old.csv'
new_ta = 'TA_new.xlsx'

find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, status_col='', path_save=ta_save, type_file='mix',kind='ta',\
                         kind_col='tenant agreement id - new',k='service type', sheetname=sheet)


New Rows:     ['CBUD1Gnot available yetJHBUD0.0Lease', 'BOJVR5not available yetBOJVR0.0Lease', '111790not available yetPRSLA0.0Lease', '170140not available yetCKDOM0.0Lease', '750430not available yetA3CHE0.0Lease', '750120not available yetA3CHU0.0Lease']
Dropped Rows: ['1610902400000063ULPOD1.0Energy', '1610902400000063ULPOD0.0Lease']

Done.



# LC Comparison


# Excel
Fit some columns before run the comparison

In [ ]:
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int, cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)

    df = pd.read_excel(path, sheet_name = sheetname,header=0, names = col_name, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        #df[col] = df[col].fillna(0)
        df[col] = df[col].apply(lambda x: '{:0,.2f}'.format(x))
    for i in bill_cols:
        df[i] = df[i].replace(['N/A', 'n/a',"0", '-', np.nan,'nan', ' '], '')

    df = df.fillna('')
    #replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df

def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    #print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

col_names = ['Contract ID - NEW', 'FIN ID', 'Contract Crncy', 'Contract Type', 'Freq.', 'Freq. Unit', 'Indexation',\
         'Index upon request', 'Counterpart ID', 'Counterpart', 'LC Amount CZK\nyearly', 'Amount in EUR yearly (Actual)', \
         'Payment terms Code', 'VAT Subject', '% of VAT', 'Contr. Start date', 'Contr. 1st End', 'Contr. Term End', \
         'Sublease Consession', 'Renewal Option', 'Unique Key', 'Count of Key', 'Count of ContrBct ', \
         'Remarks - Consistency check', 'Residual Period in years', 'Scoping classification', 'DAS sites', \
         'DAS ownership', '31_12_9999', 'Unique Key1', 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021', \
         'Check LC amount', 'Updated Item\nAmount yearly', 'Contr. 1st End as of 31.05.2021', \
         'Contr. 1st End as of 30.06.2021', 'Check Contr. 1st End', 'Updated Item\n1st End date', 'Comment', \
         'Comment date', 'Date']

#ta = pd.read_csv(path_ta_input, header=0, names=col)
dates_lc = ['Contr. Start date', 'Contr. 1st End','contr. term end', 'Contr. 1st End as of 31.05.2021', 'Contr. 1st End as of 30.06.2021']
interger_lc = []
amount_lc = ["Amount in EUR yearly (Actual)", 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021']
lc_bill_cols = ['Contract ID - NEW','Counterpart']

lc = read_files('/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx', 'Final data_Lease',col_names, 1, 0, 'FIN ID', \
                dates_lc, interger_lc,amount_lc, lc_bill_cols)

col_order_lc = ["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"]
lc = lc[col_order_lc]

change_incorret(lc, "Contract ID - NEW", '% of VAT', 0.21,'21%')

lc.head(2)

In [27]:
lc_old = pd.read_csv('/content/LC_Input_CzechRepublic_20210731.csv').fillna('')
lc_old.head(3)

def find_diffs_between_files(path_OLD, path_NEW, index_col, bill_cols, \
                             path_save,old_file,new_file, add_col='', type_file='mix', sheetname='', skipr=0, skipc=0):
    import re
    import xlsxwriter
    from openpyxl import Workbook, styles
    from openpyxl.styles import PatternFill, Font
    from openpyxl.styles.differential import DifferentialStyle
    from openpyxl.formatting.rule import Rule

    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df):
        fit_cols = lower_str(list(df.columns))
        df.columns = fit_cols
        #df = df[bill_cols]
        df = df.dropna(subset=[index_col], axis=0)
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    def number_add(df, index_col, add_col):
        df = df.sort_values(by=[index_col, add_col], ascending=[True, False])
        """lista = []
        df['sites'] = df[index_col].copy()
        for i in range(len(df[index_col])):
            if str(df.iloc[i][index_col]) in lista:            
                df.iloc[i]['sites'] = str(df.iloc[i][index_col])+"_"+str(lista.count(str(df.iloc[i][index_col])))
            else:
                df.iloc[i]['sites'] = str(df.iloc[i][index_col])+"_0"    
        lista.append(str(df_OLD.iloc[i][index_col]))"""
        lista=[]
        dico = {}
        for id in df[index_col]:
            nome = str(id)
            if nome not in dico.keys():
                dico[nome] = 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)
            else:
                dico[nome] += 1
                n = nome+f'_{dico[nome]}'
                lista.append(n)

        return lista
        

    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()

    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'mix':
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)
    else: 
        df_OLD = path_OLD
        df_OLD.columns = lower_str(list(df_OLD.columns)) 
        df_OLD[index_col] = df_OLD[index_col].astype(str)
        #df_OLD['sites'] = df_OLD[index_col]+df_OLD['counterpart']    
        df_OLD['sites'] = number_add(df_OLD, index_col, add_col)
        df_OLD = df_OLD.set_index('sites').fillna('')
        #df_OLD = fit_df(df_OLD)

        df_NEW = path_NEW
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW[index_col] = df_NEW[index_col].astype(str)
        #df_NEW['sites'] = df_NEW[index_col]+df_NEW['counterpart']
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW['sites'] = number_add(df_NEW, index_col, add_col)
        #df_NEW = fit_df(df_NEW)
        df_NEW = df_NEW.set_index('sites').fillna('')


    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    new_copy = new_copy.reset_index()
    print(new_copy)
    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save}_{old_file} vs {new_file}.xlsx'
    #file = pd.ExcelWriter(fname, engine='openpyxl')
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)
    
    """file.save()

    # get openpyxl objects
    from openpyxl import load_workbook

    wb  = load_workbook(fname)
    ws = wb['diffs_founded']"""
    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ20000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                                end_color='00FF0000', fill_type='solid')
        
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ20000', header_style)
     
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

lc_bills = ['Contract ID - NEW','Counterpart', 'LC Amount CZK\nyearly', 'Contr. Start date', 'Contr. 1st End']
lc_save = '/content/CZ_LC'
old_name = '20210731.csv'
new_name = 'VF CZ_v1.xlsx'
find_diffs_between_files(lc_old, lc, 'Contract ID - NEW', lc_bills, \
                         lc_save,old_name,new_name,'counterpart',type_file='else')

             sites  ... lc amount as of 30.04.2021
0     1100000000_1  ...                           
1     1100000001_1  ...                           
2     1100000002_1  ...                           
3     1100000006_1  ...                           
4     1100000006_2  ...                           
...            ...  ...                        ...
4674  1100007693_1  ...                           
4675  1100007696_1  ...                           
4676  1100007698_1  ...                           
4677  1100007699_1  ...                           
4678  1100007702_1  ...                           

[4679 rows x 45 columns]

New Rows:     ['1100000238_2', '1100000440_2', '1100001041_2', '1100001587_2', '1100002195_2', '1100002342_2', '1100002615_2', '1100002641_2', '1100004334_2', '1100007625_1', '1100007696_1', '1100007698_1', '1100007699_1', '1100007702_1']
Dropped Rows: ['1100002084_1', '1100004297_1', '1100004297_2']

Done.



# Old version
More difficult to run cause I need to fit a lot of columns insede de function

In [ ]:
def find_diffs_between_files_lc(path_OLD, path_NEW, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', kind_col='', k='', am_cols = [],\
                             cols_int=[], cols_date=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist
     
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format
    
    def date_parser(df, column,format, type_date):
        if format == 'mix':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
            return df[column]
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df


    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    am_cols = lower_str(am_cols)
    cols_int = lower_str(cols_int)
    cols_date = lower_str(cols_date)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,bill_cols)

        #Ajustado só para CZ_Julho, tirei o encoding
        df_NEW = pd.read_csv(path_NEW).fillna('')
        df_NEW = fit_df(df_NEW,bill_cols)

    else:
        col_order_lc = lower_str(["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                        "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                        "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                        "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                        "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                        "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                        "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                        "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"])
        df_OLD = pd.read_csv(path_OLD, header=0, names=col_order_lc)
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD = df_OLD.sort_values(by='contract id - new')
        
        lista = []
        df_OLD['sites'] = df_OLD['contract id - new'].copy()
        for i in range(len(df_OLD['contract id - new'])):
            if str(df_OLD.iloc[i]['contract id - new']) in lista:            
                df_OLD.iloc[i]['sites'] = str(df_OLD.iloc[i]['contract id - new'])+"_"+str(lista.count(str(df_OLD.iloc[i]['contract id - new'])))
            else:
                df_OLD.iloc[i]['sites'] = str(df_OLD.iloc[i]['contract id - new'])+"_0"    
            lista.append(str(df_OLD.iloc[i]['contract id - new']))   

       

        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD[k].astype(str)+df_OLD['count of contrbct '].astype(str)+df_OLD["count of key"].astype(str)+df_OLD["counterpart"].astype(str)
        df_OLD = df_OLD.fillna('')

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])

        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        #df_OLD = df_OLD.set_index('sites').fillna('')

        col_names = ['Contract ID - NEW', 'FIN ID', 'Contract Crncy', 'Contract Type', 'Freq.', 'Freq. Unit', 'Indexation',\
                'Index upon request', 'Counterpart ID', 'Counterpart', 'LC Amount CZK\nyearly', 'Amount in EUR yearly (Actual)', \
                'Payment terms Code', 'VAT Subject', '% of VAT', 'Contr. Start date', 'Contr. 1st End', 'Contr. Term End', \
                'Sublease Consession', 'Renewal Option', 'Unique Key', 'Count of Key', 'Count of ContrBct ', \
                'Remarks - Consistency check', 'Residual Period in years', 'Scoping classification', 'DAS sites', \
                'DAS ownership', '31_12_9999', 'Unique Key1', 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021', \
                'Check LC amount', 'Updated Item\nAmount yearly', 'Contr. 1st End as of 31.05.2021', \
                'Contr. 1st End as of 30.06.2021', 'Check Contr. 1st End', 'Updated Item\n1st End date', 'Comment', \
                'Comment date', 'Date']
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,header=0, names=col_names, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=col_order_lc)
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW = df_NEW.sort_values(by='contract id - new')

        df_NEW['sites'] = df_NEW[index_col].copy()
        df_NEW['sites'] = df_NEW['sites'].astype(str)
        df_NEW['sites'] = np.where(df_NEW['sites'].duplicated(keep=False), 
                      df_NEW['sites'] + df_NEW.groupby('sites').cumcount().add(1).astype(str),
                      df_NEW['sites'])
        print(df_NEW['sites'])
        #df_NEW['sites'] = [str(site)+"_"+str(i) for i, site in enumerate(df_NEW['contract id - new'])]

        #df_NEW['sites'] = df_NEW[index_col].astype(str)+df_NEW[kind_col].astype(str)+df_NEW[k].astype(str)+df_NEW['count of contrbct '].astype(str)+df_NEW["counterpart"].astype(str)+df_NEW["counterpart"].astype(str)
        
        df_NEW = replace_values(df_NEW, am_cols, 0)
        for col in am_cols:
            #df[col] = df[col].fillna(0)
            df_NEW[col] = df_NEW[col].astype(int).apply(lambda x: f'{x:,}')

        df_NEW = df_NEW.fillna('')

        df_NEW = replace_values(df_NEW,bill_cols)
        for i in cols_date:
            df_NEW[i] = date_parser(df_NEW, i, 'mix', "%d/%m/%Y")
        df_NEW = df_NEW.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

        try:
            df_NEW = df_NEW.set_index('sites').fillna('')
        except:
            print('falha')
        #print(list(df_NEW.columns)==(df_OLD.columns))

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} old_file vs new_file.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

lc_col = ['FIN ID', 'Counterpart', 'LC Amount CZK\nyearly','Contr. Start date', 'Contr. 1st End']
path_lc_input = '/content/TowerDB_FY21-06 June-Skylon VF CZ_v1.xlsx'
lc_old = '/content/LC_Input_CzechRepublic_20210731.csv'
lc_save = '/content/CZ_LC'
ta_tab = 'Final data_Lease'
dates_lc = ['Contr. Start date', 'Contr. 1st End', 'Contr. 1st End as of 31.05.2021', 'Contr. 1st End as of 30.06.2021']
interger_lc = []
amount_lc = ['LC Amount CZK\nyearly']
lc_bill_cols = ['Contract ID - NEW','Counterpart']
"""find_diffs_between_files(ta_old, path_ta_input, 'Code', ta_cols, \
                         status_col='', path_save=ta_save, type_file='mix',kind='ta',\
                         kind_col='tenant agreement id - new',k='service type',am_cols = amount_ta,\
                         cols_int=interger, cols_date=dates, sheetname=sheet)"""
find_diffs_between_files_lc(lc_old, path_lc_input, 'FIN ID', lc_col,status_col='', path_save=lc_save,type_file='mix', kind='ta',\
                            kind_col = 'contract id - new', k='contract type',am_cols=amount_lc,\
                           cols_int=[], cols_date=dates_lc, sheetname=ta_tab, skipr=1)

# CSV
Old version


In [ ]:
def find_diffs_between_files_lc(path_OLD, path_NEW,new_file, old_file, index_col, bill_cols, \
                             status_col, path_save, type_file='mix',kind='tw', kind_col='', k='', am_cols = [],\
                             cols_int=[], cols_date=[], sheetname='', skipr=0, skipc=0):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist
     
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format
    
    def date_parser(df, column,format, type_date):
        if format == 'mix':
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                        and not isinstance(date_obj, str) else date_obj for date_obj in df[column]]
            df[column] = df[column].astype(str)
            return df[column]
        else:
            df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
            df[column] = df[column].astype(str)
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df


    bill_cols = lower_str(bill_cols)
    index_col = index_col.lower()
    type_file = type_file.lower()
    status_col = status_col.lower()
    am_cols = lower_str(am_cols)
    cols_int = lower_str(cols_int)
    cols_date = lower_str(cols_date)
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = fit_df(df_OLD,bill_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,bill_cols)

    elif type_file == 'csv':
        col_order_lc = ["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                        "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                        "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                        "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                        "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                        "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                        "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                        "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"]
        col_order_lc = lower_str(col_order_lc)
        df_OLD = pd.read_csv(path_OLD, header=0, names=col_order_lc, encoding='latin2')
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD = df_OLD.sort_values(by='contract id - new')
        
        lista = []

        df_OLD['sites'] = df_OLD['contract id - new'].copy()
        for i in range(len(df_OLD['contract id - new'])):
            if str(df_OLD.iloc[i]['contract id - new']) in lista:           
                df_OLD.loc[i, 'sites'] = str(df_OLD.iloc[i]['contract id - new'])+"_"+str(lista.count(str(df_OLD.iloc[i]['contract id - new'])))
            else:  
                df_OLD.loc[i, 'sites']= str(df_OLD.iloc[i]['contract id - new'])+"_0"

            lista.append(str(df_OLD.iloc[i]['contract id - new']))   

        print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])


        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD[k].astype(str)+df_OLD['count of contrbct '].astype(str)+df_OLD["count of key"].astype(str)+df_OLD["counterpart"].astype(str)
        df_OLD = df_OLD.fillna('')
        #df_OLD = df_OLD.apply(unidecode)

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_OLD = df_OLD.set_index('sites').fillna('')
        
        """New"""

        df_NEW = pd.read_csv(path_NEW, header=0, names=col_order_lc, encoding='latin2')
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_NEW = df_NEW.sort_values(by='contract id - new')
        
        lista = []

        df_NEW['sites'] = df_NEW['contract id - new'].copy()
        for i in range(len(df_NEW['contract id - new'])):
            if str(df_NEW.iloc[i]['contract id - new']) in lista:           
                df_NEW.loc[i, 'sites'] = str(df_NEW.iloc[i]['contract id - new'])+"_"+str(lista.count(str(df_NEW.iloc[i]['contract id - new'])))
            else:  
                df_NEW.loc[i, 'sites']= str(df_NEW.iloc[i]['contract id - new'])+"_0"

            lista.append(str(df_NEW.iloc[i]['contract id - new']))   

        print([i for i in list(df_NEW['sites']) if list(df_NEW['sites']).count(i)>1])


        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD[k].astype(str)+df_OLD['count of contrbct '].astype(str)+df_OLD["count of key"].astype(str)+df_OLD["counterpart"].astype(str)
        df_NEW = df_NEW.fillna('')
        #df_OLD = df_OLD.apply(unidecode)

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_NEW = df_NEW.set_index('sites').fillna('')

        #Ajustado só para CZ_Julho, tirei o encoding


    else:
        col_order_lc = ["Contract ID - NEW","FIN ID","Contract Crncy","Contract Type","Freq.","Freq. Unit","Indexation",\
                        "Index upon request","Counterpart ID","Counterpart","LC Amount CZK\nyearly","Amount in EUR yearly (Actual)",\
                        "Payment terms Code","VAT Subject","% of VAT","Contr. Start date","Contr. 1st End","Contr. Term End",\
                        "Sublease Consession","Renewal Option","Unique Key","Count of Key","Count of ContrBct ",\
                        "Remarks - Consistency check","Residual Period in years","Scoping classification","DAS sites",\
                        "DAS ownership","31_12_9999","Unique Key1",'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021',\
                        "Check LC amount","Updated Item\nAmount yearly",'Contr. 1st End as of 31.05.2021','Contr. 1st End as of 30.06.2021',\
                        "Check Contr. 1st End","Updated Item\n1st End date","Comment","Comment date","Date"]
        col_order_lc = lower_str(col_order_lc)
        df_OLD = pd.read_csv(path_OLD, header=0, names=col_order_lc, encoding='latin2')
        #df_OLD = fit_df(df_OLD,bill_cols)
        df_OLD.columns = lower_str(list(df_OLD.columns))
        df_OLD = df_OLD.dropna(subset=[index_col], axis=0)
        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        df_OLD = df_OLD.sort_values(by='contract id - new')
        
        lista = []

        df_OLD['sites'] = df_OLD['contract id - new'].copy()
        for i in range(len(df_OLD['contract id - new'])):
            if str(df_OLD.iloc[i]['contract id - new']) in lista:           
                df_OLD.loc[i, 'sites'] = str(df_OLD.iloc[i]['contract id - new'])+"_"+str(lista.count(str(df_OLD.iloc[i]['contract id - new'])))
            else:  
                df_OLD.loc[i, 'sites']= str(df_OLD.iloc[i]['contract id - new'])+"_0"

            lista.append(str(df_OLD.iloc[i]['contract id - new']))   

        print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])


        #df_OLD['sites'] = df_OLD[index_col].astype(str)+df_OLD[kind_col].astype(str)+df_OLD[k].astype(str)+df_OLD['count of contrbct '].astype(str)+df_OLD["count of key"].astype(str)+df_OLD["counterpart"].astype(str)
        df_OLD = df_OLD.fillna('')
        #df_OLD = df_OLD.apply(unidecode)

        #print([i for i in list(df_OLD['sites']) if list(df_OLD['sites']).count(i)>1])
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df_OLD = df_OLD.set_index('sites').fillna('')

        col_names = ['Contract ID - NEW', 'FIN ID', 'Contract Crncy', 'Contract Type', 'Freq.', 'Freq. Unit', 'Indexation',\
                'Index upon request', 'Counterpart ID', 'Counterpart', 'LC Amount CZK\nyearly', 'Amount in EUR yearly (Actual)', \
                'Payment terms Code', 'VAT Subject', '% of VAT', 'Contr. Start date', 'Contr. 1st End', 'Contr. Term End', \
                'Sublease Consession', 'Renewal Option', 'Unique Key', 'Count of Key', 'Count of ContrBct ', \
                'Remarks - Consistency check', 'Residual Period in years', 'Scoping classification', 'DAS sites', \
                'DAS ownership', '31_12_9999', 'Unique Key1', 'LC amount as of 31.05.2021', 'LC amount as of 30.06.2021', \
                'Check LC amount', 'Updated Item\nAmount yearly', 'Contr. 1st End as of 31.05.2021', \
                'Contr. 1st End as of 30.06.2021', 'Check Contr. 1st End', 'Updated Item\n1st End date', 'Comment', \
                'Comment date', 'Date']
        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,header=0, names=col_names, skiprows = skipr)
        df_NEW.columns = lower_str(list(df_NEW.columns))
        df_NEW = df_NEW.reindex(columns=col_order_lc)
        df_NEW = df_NEW.dropna(subset=[index_col], axis=0)
        df_NEW = df_NEW.sort_values(by='contract id - new')

        lista = []
        df_NEW['sites'] = df_NEW['contract id - new'].copy()
        for i in range(len(df_NEW['contract id - new'])):
            if str(df_NEW.iloc[i]['contract id - new']) in lista:           
                df_NEW.loc[i, 'sites'] = str(df_NEW.iloc[i]['contract id - new'])+"_"+str(lista.count(str(df_NEW.iloc[i]['contract id - new'])))
            else:   
                df_NEW.loc[i, 'sites'] = str(df_NEW.iloc[i]['contract id - new'])+"_0"

            lista.append(str(df_NEW.iloc[i]['contract id - new']))   

        #print([i for i in list(df_NEW['sites']) if list(df_NEW['sites']).count(i)>1])
        df_NEW = replace_values(df_NEW, am_cols, 0)
        for col in am_cols:
            #df[col] = df[col].fillna(0)
            df_NEW[col] = df_NEW[col].astype(int).apply(lambda x: f'{x:,}')

        df_NEW = df_NEW.fillna('')

        df_NEW = replace_values(df_NEW,bill_cols)
        for i in cols_date:
            df_NEW[i] = date_parser(df_NEW, i, 'mix', "%d/%m/%Y")
        df_NEW = df_NEW.applymap(lambda x: x.encode('unicode_escape').decode('ISO-8859-1') if isinstance(x, str) else x)
        #print(df_NEW.info())
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos

        try:
            df_NEW = df_NEW.set_index('sites').fillna('')
        except:
            print('falha')
        #print(list(df_NEW.columns)==(df_OLD.columns))

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))

    for row in new_copy.index:

        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0, how='all')
    new_copy = new_copy.dropna(axis=1, how='all')

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()
    
    n = 10
    lista = []
    for site in new_copy['sites']:
        chunks = [site[i:i+n] for i in range(0, len(site), n)]
        lista.append(chunks[0])
    new_copy['sites_code'] = lista
    new_copy = new_copy[['sites_code']+ new_copy.columns[:-1].tolist()]

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} {old_file} vs {new_file}.xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ10000', highlight)
    
    red_header = PatternFill(start_color='00FF0000', \
                             end_color='00FF0000', fill_type='solid')
    for column in bill_cols:
        dx = DifferentialStyle(font=Font(color='FFFFFF', bold=True), fill=red_header)
        header_style = Rule(type="containsText", operator="containsText", text=column, dxf=dx)
        header_style.formula = [f'SEARCH("{column}", A1)']
        ws.conditional_formatting.add('A1:ZZ10000', header_style)

    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy['sites']==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    ws.delete_cols(2)
    wb.save(fname)
    print('\nDone.\n')

lc_col = ['FIN ID', 'Counterpart', 'LC Amount CZK\nyearly','Contr. Start date', 'Contr. 1st End']
path_lc_input = '/content/LC_Input_CzechRepublic_20210831.csv'
lc_old = '/content/LC_Input_CzechRepublic_20210630_True-up.csv'
lc_save = '/content/CZ_LC'
lc_tab = 'Final data_Lease'
dates_lc = ['Contr. Start date', 'Contr. 1st End', 'Contr. 1st End as of 31.05.2021', 'Contr. 1st End as of 30.06.2021']
interger_lc = []
amount_lc = ['LC Amount CZK\nyearly']
lc_bill_cols = ['Contract ID - NEW','Counterpart']
lc_nn = 'celfinet_20210831.csv'
lc_on = 'LC_20210630_True-up.csv'
find_diffs_between_files_lc(lc_old, path_lc_input,lc_nn,lc_on, 'FIN ID', lc_col,status_col='', path_save=lc_save,type_file='csv', kind='ta',\
                            kind_col = 'contract id - new', k='contract type',am_cols=amount_lc,\
                           cols_int=[], cols_date=dates_lc, sheetname=lc_tab, skipr=1)

[]
[]

New Rows:     []
Dropped Rows: []

Done.



# UIS Comparison

In [ ]:
def find_diffs_between_files_uis(path_OLD, path_NEW, index_col, uis_cols, \
                             path_save, old_name, new_name, type_file='mix', sheetname='', skipr=0, skipc=0):
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def fit_df(df, uis_cols):

        df = df.dropna(subset=[index_col], axis=0)
        #df = df[uis_cols]
        #df = df.drop_duplicates(subset=[index_col], keep='last')
        df[index_col] = df[index_col].astype(str)
        df['sites'] = df[index_col]
        # Aqui ajusta os nan e nat dos DFs, quando não forem arquivos não lidos
        df = df.set_index('sites').fillna('')
        return df
        
    def change_format(index, worksheet, format):
        for cell in worksheet[f"{str(index)}:{str(index)}"]:
            cell.font = format

    #uis_cols = lower_str(uis_cols)
    #index_col = index_col.lower()
    # type_file = type_file.lower()
    if type_file == 'excel':
        df_OLD = pd.read_excel(path_OLD, sheet_name = sheetname,  skiprows = skipr).fillna('')
        df_OLD = df_OLD.iloc[:,skipc:]
        df_OLD = df_OLD.drop([0,0])
        df_OLD = fit_df(df_OLD,uis_cols)


        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname,  skiprows = skipr).fillna('')
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = df_NEW.drop([0,0]) 
        df_NEW = fit_df(df_NEW,uis_cols)


    elif type_file == 'csv':
        df_OLD = pd.read_csv(path_OLD, encoding='latin').fillna('')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_csv(path_NEW, encoding='latin').fillna('')
        df_NEW = fit_df(df_NEW,uis_cols)

    else:
        df_OLD = pd.read_csv(path_OLD, encoding='latin')
        df_OLD = fit_df(df_OLD,uis_cols)

        df_NEW = pd.read_excel(path_NEW,sheet_name = sheetname, skiprows = skipr)
        df_NEW = df_NEW.iloc[:,skipc:]
        df_NEW = fit_df(df_NEW,uis_cols)

    # Perform Diff
    new_copy = df_NEW.copy()
    droppedRows = []
    newRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    print(len(sharedCols))
    print(sharedCols)
    for row in new_copy.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = str(df_OLD.loc[row,col])
                value_NEW = str(df_NEW.loc[row,col])
               #Error de a.empty() são sites duplcados e nã poodem estar no index
                if value_OLD == value_NEW:
                    new_copy.loc[row,col] = np.nan
                else:
                    new_copy.loc[row,col] = f'{value_OLD} > {value_NEW}'
        else:
            newRows.append(row)

    new_copy = new_copy.dropna(axis=0,thresh=len(new_copy.columns[1:]))
    new_copy = new_copy.dropna(axis=1)
    print(new_copy)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            new_copy = new_copy.append(df_OLD.loc[row,:])
    
    new_copy = new_copy.sort_index(key=lambda x: x.str.lower()).fillna('')
    
    new_copy = new_copy.reset_index()

    print(f'\nNew Rows:     {newRows}')
    print(f'Dropped Rows: {droppedRows}')

    # Save output and format
    fname = f'{path_save} - [{old_name}] vs [{new_name}].xlsx'
    file = pd.ExcelWriter(fname, engine='openpyxl')
    
    new_copy.to_excel(file, sheet_name='diffs_founded', index=False)
    df_NEW.to_excel(file, sheet_name='new_file', index=False)
    df_OLD.to_excel(file, sheet_name='old_file', index=False)

    # get openpyxl objects
    wb  = file.book
    ws = file.sheets['diffs_founded']
    
    red_fill = PatternFill(start_color='95A7B3', \
                                end_color='95A7B3', fill_type='solid')
    red_font = Font(color='00FF0000', italic=True)
    new_fmt = Font(color='3F976D',bold=True, italic=True)
    removed = Font(color='95A7B3',bold=True, italic=True)

    dxf = DifferentialStyle(font=red_font, fill=red_fill)
    highlight = Rule(type="containsText", operator="containsText", text=">", dxf=dxf)
    highlight.formula = ['SEARCH(">", A1)']
    ws.conditional_formatting.add('A1:ZZ1000', highlight)
    
    #print(len(newRows))
    for site in new_copy['sites']:
        if site in newRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,new_fmt)
        if site in droppedRows:
            idx = new_copy.index[new_copy[index_col]==site].to_list()
            idx = list(map(lambda x: x+2, idx))
            change_format(*idx,ws,removed)
    
    wb.save(fname)
    print('\nDone.\n')

col_uis = ['Site_ID (Alphanumeric, Alphabetical or Numeric)',
       'Site Categorization as Phase 1 or Phase 2 Site',
       'BTS site applicable charge (Annual)',
       'Commercials for sites beyond 10% cap of critical sites (Annual)',
       'Total charges to be applied for subsequent active sharing arrangement',
       'Fixed component of energy bill', 'Variable component of energy bill',
       'Energy consumption (for metered site)',
       'Agreed unit price (for metered sites)',
       'Operator’s active energy charge through estimated model',
       'Capex incurred by TowerCo under clause 29 to be reimbursed by Operator',
       'Energy reduction measures - Final value to be charged',
       'Legacy site upgrades Or BBU capacity upgrade - Final value to be charged',
       'BTS site - Site service order withdrawal - Final value to be charged',
       'Energy infrastructure enhancements - Final value to be charged',
       'One-off power related costs - Final value to be charged',
       'Space blocking - Unutilized sites\nIf unutilized then select Yes ',
       'Decommissioning charges - Equipment removal cost - Final value to be charged',
       'Site exit fee - Site ID part of delta',
       'Site exit fee - Remaining term of site IDs in delta',
       'Site power availability - Site IDs where service credits are applicable',
       'Site power availability - Total hours applicable ',
       'Site power availability - Unavailable time',
       'Site power availability - Count of repeat incidents (if applicable)',
       'Site cooling - Final value associated with Service Credit applicable',
       'Site access - Final value associated with Service Credit applicable',
       'Incident resolution - Final value associated with Service Credit applicable',
       'BTS sites - Deployment time - Delay',
       'Site modification Completion time - Delay',
       'Delay In RSCR site modification completion Time',
       'Delay In RSCR BTS site completion time',
       'Additional Customer Discount percentage applicable (Number only)',
       'Delay in Site Modification Projects', 'Delay in BTS Projects',
       'Excess of Upgrade Capital Expenditure over Threshold']
       
pathuis_n = '/content/UserInput_CzechRepublic_20210831.xlsx'
pathuis_o = '/content/UserInput_CzechRepublic_20210630 TrueUp.xlsx'
sheetuis='SiteLevel'
skiprows=2
to_save = '/content/CZ_UIS'
old_n = 'UIS - 20210630 TrueUp.xlsx'
new_n = "UIS - 20210831.xlsx"

find_diffs_between_files_uis(pathuis_o, pathuis_n, 'Site_ID (Alphanumeric, Alphabetical or Numeric)', col_uis, \
                        to_save, old_n, new_n, type_file='excel', sheetname=sheetuis, skipr=2)